# About

In this programming assignment you will train a classifier to identify type of a particle. There are six particle types: electron, proton, muon, kaon, pion and ghost. Ghost is a particle with other type than the first five or a detector noise. 

Different particle types remain different responses in the detector systems or subdetectors. Thre are five systems: tracking system, ring imaging Cherenkov detector (RICH), electromagnetic and hadron calorimeters, and muon system.

![pid](pic/pid.jpg)

You task is to identify a particle type using the responses in the detector systems. 

# Attention

Data files you should download from https://github.com/hse-aml/hadron-collider-machine-learning/releases/tag/Week_2

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import utils

# Download data

Download data used to train classifiers.

### Read training file

In [2]:
data = pandas.read_csv('training.csv.gz')

In [3]:
data.head()

,TrackP,TrackNDoFSubdetector2,BremDLLbeElectron,MuonLooseFlag,FlagSpd,SpdE,EcalDLLbeElectron,DLLmuon,RICHpFlagElectron,EcalDLLbeMuon,...,TrackNDoF,RICHpFlagMuon,RICH_DLLbeKaon,RICH_DLLbeElectron,HcalE,MuonFlag,FlagMuon,PrsE,RICH_DLLbeMuon,RICH_DLLbeProton
0,74791.156263,15.0,0.232275,1.0,1.0,3.2,-2.505719,6.604153,1.0,1.929960,...,28.0,1.0,-7.213300,-0.280200,5586.589846,1.0,1.0,10.422315,-2.081143e-07,-24.824400
1,2738.489989,15.0,-0.357748,0.0,1.0,3.2,1.864351,0.263651,1.0,-2.061959,...,32.0,1.0,-0.324317,1.707283,-0.000007,0.0,1.0,43.334935,2.771583e+00,-0.648017
2,2161.409908,17.0,-999.000000,0.0,0.0,-999.0,-999.000000,-999.000000,0.0,-999.000000,...,27.0,0.0,-999.000000,-999.000000,-999.000000,0.0,0.0,-999.000000,-9.990000e+02,-999.000000
3,15277.730490,20.0,-0.638984,0.0,1.0,3.2,-2.533918,-8.724949,1.0,-3.253981,...,36.0,1.0,-35.202221,-14.742319,4482.803707,0.0,1.0,2.194175,-3.070819e+00,-29.291519
4,7563.700195,19.0,-0.638962,0.0,1.0,3.2,-2.087146,-7.060422,1.0,-0.995816,...,33.0,1.0,25.084287,-10.272412,5107.554680,0.0,1.0,0.000015,-5.373712e+00,23.653087


### List of columns in the samples

Here, **Spd** stands for Scintillating Pad Detector, **Prs** - Preshower, **Ecal** - electromagnetic calorimeter, **Hcal** - hadronic calorimeter, **Brem** denotes traces of the particles that were deflected by detector.

- ID - id value for tracks (presents only in the test file for the submitting purposes)
- Label - string valued observable denoting particle types. Can take values "Electron", "Muon", "Kaon", "Proton", "Pion" and "Ghost". This column is absent in the test file.
- FlagSpd - flag (0 or 1), if reconstructed track passes through Spd
- FlagPrs - flag (0 or 1), if reconstructed track passes through Prs
- FlagBrem - flag (0 or 1), if reconstructed track passes through Brem
- FlagEcal - flag (0 or 1), if reconstructed track passes through Ecal
- FlagHcal - flag (0 or 1), if reconstructed track passes through Hcal
- FlagRICH1 - flag (0 or 1), if reconstructed track passes through the first RICH detector
- FlagRICH2 - flag (0 or 1), if reconstructed track passes through the second RICH detector
- FlagMuon - flag (0 or 1), if reconstructed track passes through muon stations (Muon)
- SpdE - energy deposit associated to the track in the Spd
- PrsE - energy deposit associated to the track in the Prs
- EcalE - energy deposit associated to the track in the Hcal
- HcalE - energy deposit associated to the track in the Hcal
- PrsDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Prs
- BremDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Brem
- TrackP - particle momentum
- TrackPt - particle transverse momentum
- TrackNDoFSubdetector1  - number of degrees of freedom for track fit using hits in the tracking sub-detector1
- TrackQualitySubdetector1 - chi2 quality of the track fit using hits in the tracking sub-detector1
- TrackNDoFSubdetector2 - number of degrees of freedom for track fit using hits in the tracking sub-detector2
- TrackQualitySubdetector2 - chi2 quality of the track fit using hits in the  tracking sub-detector2
- TrackNDoF - number of degrees of freedom for track fit using hits in all tracking sub-detectors
- TrackQualityPerNDoF - chi2 quality of the track fit per degree of freedom
- TrackDistanceToZ - distance between track and z-axis (beam axis)
- Calo2dFitQuality - quality of the 2d fit of the clusters in the calorimeter 
- Calo3dFitQuality - quality of the 3d fit in the calorimeter with assumption that particle was electron
- EcalDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Ecal
- EcalDLLbeMuon - delta log-likelihood for a particle candidate to be muon using information from Ecal
- EcalShowerLongitudinalParameter - longitudinal parameter of Ecal shower
- HcalDLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from Hcal
- HcalDLLbeMuon - delta log-likelihood for a particle candidate to be using information from Hcal
- RICHpFlagElectron - flag (0 or 1) if momentum is greater than threshold for electrons to produce Cherenkov light
- RICHpFlagProton - flag (0 or 1) if momentum is greater than threshold for protons to produce Cherenkov light
- RICHpFlagPion - flag (0 or 1) if momentum is greater than threshold for pions to produce Cherenkov light
- RICHpFlagKaon - flag (0 or 1) if momentum is greater than threshold for kaons to produce Cherenkov light
- RICHpFlagMuon - flag (0 or 1) if momentum is greater than threshold for muons to produce Cherenkov light
- RICH_DLLbeBCK  - delta log-likelihood for a particle candidate to be background using information from RICH
- RICH_DLLbeKaon - delta log-likelihood for a particle candidate to be kaon using information from RICH
- RICH_DLLbeElectron - delta log-likelihood for a particle candidate to be electron using information from RICH
- RICH_DLLbeMuon - delta log-likelihood for a particle candidate to be muon using information from RICH
- RICH_DLLbeProton - delta log-likelihood for a particle candidate to be proton using information from RICH
- MuonFlag - muon flag (is this track muon) which is determined from muon stations
- MuonLooseFlag muon flag (is this track muon) which is determined from muon stations using looser criteria
- MuonLLbeBCK - log-likelihood for a particle candidate to be not muon using information from muon stations
- MuonLLbeMuon - log-likelihood for a particle candidate to be muon using information from muon stations
- DLLelectron - delta log-likelihood for a particle candidate to be electron using information from all subdetectors
- DLLmuon - delta log-likelihood for a particle candidate to be muon using information from all subdetectors
- DLLkaon - delta log-likelihood for a particle candidate to be kaon using information from all subdetectors
- DLLproton - delta log-likelihood for a particle candidate to be proton using information from all subdetectors
- GhostProbability - probability for a particle candidate to be ghost track. This variable is an output of classification model used in the tracking algorithm.

Delta log-likelihood in the features descriptions means the difference between log-likelihood for the mass hypothesis that a given track is left by some particle (for example, electron) and log-likelihood for the mass hypothesis that a given track is left by a pion (so, DLLpion = 0 and thus we don't have these columns). This is done since most tracks (~80%) are left by pions and in practice we actually need to discriminate other particles from pions. In other words, the null hypothesis is that particle is a pion.

### Look at the labels set

The training data contains six classes. Each class corresponds to a particle type. Your task is to predict type of a particle.

In [4]:
set(data.Label)

{'Electron', 'Ghost', 'Kaon', 'Muon', 'Pion', 'Proton'}

Convert the particle types into class numbers.

In [5]:
data['Class'] = utils.get_class_ids(data.Label.values)
set(data.Class)

{0, 1, 2, 3, 4, 5}

### Define training features

The following set of features describe particle responses in the detector systems:

![features](pic/features.jpeg)

Also there are several combined features. The full list is following.

In [6]:
features = list(set(data.columns) - {'Label', 'Class'})
features

['MuonLooseFlag',
 'FlagSpd',
 'FlagPrs',
 'MuonLLbeMuon',
 'PrsDLLbeElectron',
 'DLLmuon',
 'TrackNDoFSubdetector1',
 'EcalE',
 'TrackNDoF',
 'FlagMuon',
 'RICH_DLLbeBCK',
 'EcalDLLbeMuon',
 'TrackDistanceToZ',
 'PrsE',
 'FlagHcal',
 'Calo2dFitQuality',
 'RICHpFlagPion',
 'TrackQualitySubdetector1',
 'FlagRICH2',
 'RICH_DLLbeMuon',
 'FlagRICH1',
 'RICHpFlagProton',
 'RICH_DLLbeProton',
 'TrackQualityPerNDoF',
 'MuonLLbeBCK',
 'RICHpFlagKaon',
 'FlagEcal',
 'RICHpFlagMuon',
 'TrackP',
 'MuonFlag',
 'HcalE',
 'FlagBrem',
 'DLLproton',
 'GhostProbability',
 'DLLkaon',
 'BremDLLbeElectron',
 'HcalDLLbeMuon',
 'HcalDLLbeElectron',
 'RICH_DLLbeKaon',
 'RICHpFlagElectron',
 'DLLelectron',
 'RICH_DLLbeElectron',
 'TrackPt',
 'TrackNDoFSubdetector2',
 'EcalDLLbeElectron',
 'EcalShowerLongitudinalParameter',
 'Calo3dFitQuality',
 'SpdE',
 'TrackQualitySubdetector2']

### Divide training data into 2 parts

In [7]:
training_data, validation_data = train_test_split(data, random_state=11, train_size=0.90, test_size=0.10)

In [8]:
len(training_data), len(validation_data)

(1080000, 120000)

In [9]:
training_data.head()

,TrackP,TrackNDoFSubdetector2,BremDLLbeElectron,MuonLooseFlag,FlagSpd,SpdE,EcalDLLbeElectron,DLLmuon,RICHpFlagElectron,EcalDLLbeMuon,...,RICHpFlagMuon,RICH_DLLbeKaon,RICH_DLLbeElectron,HcalE,MuonFlag,FlagMuon,PrsE,RICH_DLLbeMuon,RICH_DLLbeProton,Class
968874,12192.330089,17.0,-0.170024,0.0,1.0,0.0,-1.831868,-0.996012,1.0,5.017775e-01,...,1.0,-19.783108,-6.720408,1914.190913,1.0,1.0,1.967353e-06,-2.804208,-25.041909,1
912808,40379.488299,7.0,-999.000000,0.0,0.0,-999.0,-999.000000,-0.792678,1.0,-9.990000e+02,...,1.0,-14.755493,-2.304893,-999.000000,0.0,1.0,-9.990000e+02,-0.967993,-18.450494,4
30045,5294.729989,12.0,-0.381504,1.0,1.0,3.2,-0.589851,-2.054347,1.0,-2.142670e+00,...,1.0,13.137106,-7.920494,2567.385966,1.0,1.0,7.569901e+01,-1.713094,13.416006,3
313784,4645.819821,17.0,-0.417099,1.0,1.0,3.2,0.649028,-8.301831,1.0,7.650223e-07,...,1.0,-22.006791,-24.447692,256.265351,0.0,1.0,1.919901e+00,-7.184691,-17.656691,1
100864,3100.629887,17.0,-0.553602,0.0,1.0,3.2,-2.723420,-12.468398,1.0,-1.438773e+00,...,1.0,0.807007,-50.220193,0.000011,0.0,1.0,-8.050605e-07,-9.541493,0.771707,2


In [10]:
training_data[features].describe()

,MuonLooseFlag,FlagSpd,FlagPrs,MuonLLbeMuon,PrsDLLbeElectron,DLLmuon,TrackNDoFSubdetector1,EcalE,TrackNDoF,FlagMuon,...,RICHpFlagElectron,DLLelectron,RICH_DLLbeElectron,TrackPt,TrackNDoFSubdetector2,EcalDLLbeElectron,EcalShowerLongitudinalParameter,Calo3dFitQuality,SpdE,TrackQualitySubdetector2
count,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,...,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06,1.080000e+06
mean,1.908407e-01,8.530556e-01,8.510917e-01,-8.087499e+02,-1.500610e+02,-1.268219e+01,6.198936e+00,2.346141e+03,2.942315e+01,8.207509e-01,...,9.514204e-01,-1.403302e+01,-5.109546e+01,9.282852e+02,1.475204e+01,-1.829481e+02,-4.257734e+02,1.003561e+03,-1.445036e+02,1.609497e+01
std,3.929640e-01,3.540507e-01,3.559983e-01,3.920690e+02,3.550281e+02,1.049807e+02,3.115005e+00,5.709488e+03,6.026995e+00,3.835609e-01,...,2.149877e-01,1.048488e+02,2.152222e+02,1.605476e+03,4.037619e+00,3.855809e+02,5.245722e+03,2.408174e+03,3.546216e+02,7.769702e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,-9.990000e+02,-9.990000e+02,-9.990000e+02,1.000000e+00,-9.990000e+02,7.000000e+00,0.000000e+00,...,0.000000e+00,-9.990000e+02,-9.990000e+02,1.447011e+00,1.000000e+00,-9.990000e+02,-3.861698e+05,-9.990000e+02,-9.990000e+02,3.229625e-04
25%,0.000000e+00,1.000000e+00,1.000000e+00,-9.990000e+02,-3.062679e+00,-5.711785e+00,4.000000e+00,-3.348990e-06,2.600000e+01,1.000000e+00,...,1.000000e+00,-7.059118e+00,-1.309732e+01,2.684007e+02,1.300000e+01,-3.000639e+00,-9.990000e+02,-9.990000e+02,0.000000e+00,1.071093e+01
50%,0.000000e+00,1.000000e+00,1.000000e+00,-9.990000e+02,-2.730653e+00,-1.985259e+00,6.000000e+00,6.599040e+02,3.000000e+01,1.000000e+00,...,1.000000e+00,-4.360961e+00,-2.355985e+00,5.460906e+02,1.600000e+01,-2.315904e+00,-2.955909e+02,1.286276e+02,3.200000e+00,1.524483e+01
75%,0.000000e+00,1.000000e+00,1.000000e+00,-9.990000e+02,-1.331085e+00,1.669644e+00,8.000000e+00,3.001512e+03,3.400000e+01,1.000000e+00,...,1.000000e+00,2.593773e-03,1.672999e+00,1.110806e+03,1.800000e+01,5.918626e-01,1.550091e+02,1.657563e+03,3.200000e+00,2.041093e+01
max,1.000000e+00,1.000000e+00,1.000000e+00,4.311586e-05,3.463160e+00,1.471079e+01,2.400000e+01,3.156805e+05,5.200000e+01,1.000000e+00,...,1.000000e+00,1.769270e+01,1.861542e+02,4.271570e+05,3.000000e+01,4.341298e+00,7.293874e+05,9.999937e+03,3.200000e+00,1.048634e+02


In [11]:
training_data[features].isnull().any().any()

False

In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
training_scale = scaler.fit_transform(training_data[features].values)

In [13]:
validation_scale = scaler.transform(validation_data[features].values)

# Sklearn classifier

On this step your task is to train **Sklearn** classifier to provide lower **log loss** value.


TASK: your task is to tune the classifier parameters to achieve the lowest **log loss** value on the validation sample you can.

In [14]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
%%time 
gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=50, subsample=0.8, random_state=13,
                                min_samples_leaf=1, max_depth=3)
gb.fit(training_scale, training_data.Class.values)

### Log loss on the cross validation sample

In [16]:
# predict each track
proba_gb = gb.predict_proba(validation_scale)

AttributeError: 'NoneType' object has no attribute 'tree_'

In [ ]:
log_loss(validation_data.Class.values, proba_gb)

# Keras neural network

On this step your task is to train **Keras** NN classifier to provide lower **log loss** value.


TASK: your task is to tune the classifier parameters to achieve the lowest **log loss** value on the validation sample you can. Data preprocessing may help you to improve your score.

In [ ]:
from keras.layers.core import Dense, Activation
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def nn_model(input_dim):
    model = Sequential()
    model.add(Dense(100, input_dim=input_dim))
    model.add(Activation('relu'))
    
    model.add(Dense(50))
    model.add(Activation('relu'))

    model.add(Dense(6))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    return model

In [ ]:
callback = [EarlyStopping(monitor='val_loss', min_delta=0, patience=4, verbose=0, mode='auto'),
            ModelCheckpoint('output/{val_loss:.4f}.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='auto')]

In [ ]:
nn = nn_model(len(features))
nn.fit(training_scale, np_utils.to_categorical(training_data.Class.values),
       validation_data=(validation_scale, np_utils.to_categorical(validation_data.Class.values)),
       epochs=50, verbose=1, batch_size=256, callbacks=callback)

In [ ]:
nn.load_weights('output/0.5680.hdf5')

### Log loss on the cross validation sample

In [ ]:
# predict each track
proba_nn = nn.predict_proba(validation_scale)

In [ ]:
log_loss(validation_data.Class.values, proba_nn)

# Quality metrics

Plot ROC curves and signal efficiency dependece from particle mometum and transverse momentum values.

In [ ]:
proba = proba_nn

In [ ]:
utils.plot_roc_curves(proba, validation_data.Class.values)

In [ ]:
utils.plot_signal_efficiency_on_p(proba, validation_data.Class.values, validation_data.TrackP.values, 60, 50)
plt.show()

In [ ]:
utils.plot_signal_efficiency_on_pt(proba, validation_data.Class.values, validation_data.TrackPt.values, 60, 50)
plt.show()

# Prepare submission

Select your best classifier and prepare submission file.

In [ ]:
test = pandas.read_csv('test.csv.gz')

In [ ]:
best_model = nn

In [ ]:
test_scale = scaler.transform(test[features])

In [ ]:
# predict test sample
submit_proba = best_model.predict_proba(test_scale)
submit_ids = test.ID

In [ ]:
from IPython.display import FileLink
utils.create_solution(submit_ids, submit_proba, filename='submission_file.csv.gz')